# Data analysis and processing

We're going to re-analyse the data from the paper [Compact, low-threshold squeezed light source](https://opg.optica.org/abstract.cfm?uri=oe-27-26-37877), as well as data taken during the project that led to the paper [Distributed quantum sensing in a continuous-variable entangled network](https://www.nature.com/articles/s41567-019-0743-x).

Both of these papers are about [squeezed light](https://omni.wikiwand.com/en/articles/Squeezed_states_of_light) generated by an [optical parametric oscillator (OPO)](https://omni.wikiwand.com/en/articles/Optical_parametric_oscillator). In the textbook description of squeezed vacuum, the variance of the squeezed quadrature (let's say it's $\hat x$) is $\text{Var}(x) = e^{-2r}$ and for the anti-squeezed quadrature ($\hat p$) it is $\text{Var}(p) = e^{2r}$. Here, $r$ is the _squeezing parameter_. For a realistic state, the situation is more complex - $r$ is not well-defined in the experiment, but the variance will instead depend on losses, the frequency that it is observed at, etc.

Denoting by $V_+$ the variance of the anti-squeezing relative to shotnoise (the standard noise level, whose value depends on the definition of the quadrature operators), and by $V_-$ the squeezing variance, the simplest formula for the squeezing of an OPO is for the case of no losses and at DC (i.e. frequency = zero):

$V_\pm = 1 \pm \frac{4x}{(1\mp x)^2},$

where $x=\sqrt{P/P_{th}}$ is the _pump parameter_ given by the power $P$ of the field used for pumping the nonlinear interaction inside the OPO relative to a _threshold power_ $P_{th}$.

For non-zero losses, we have the _overall efficiency_ $\eta = 1-\text{losses}$, and we also introduce the _OPO half-width-half-max bandwidth_ $f_{HWHM}$ and the frequency of observation $f$. Then the expression above becomes

$V_\pm = 1 \pm \frac{4x\eta}{(1\mp x)^2 + (f/f_{HWHM})^2}.$

The variance of a general quadrature $x_\theta$ at an angle $\theta$ with the squeezed quadrature can be obtained by

$V(\theta) = V_-\cos^2\theta + V_+\sin^2\theta$.

Finally, if there is some fluctuation in the observed phase, this can be incorporated by modeling it as a normal distribution around the phase set point with standard deviation $\sigma$, resulting in a variance

$V(\theta) = \frac{1}{2} V_- (1+\cos(2\theta)e^{-2\sigma^2}) + \frac{1}{2} V_+ (1-\cos(2\theta)e^{-2\sigma^2}) .$

#### Data loading and processing

The data for the first paper (the "Jens" paper) has been acquired by a Keysight N9000A signal analyzer. It is in CSV format which is easy to import e.g. with NumPy's `genfromtxt` function.

The data for the second paper (the "Xueshi" paper) has been acquired by a LeCroy HDO6034 oscilloscope. This is in a proprietary binary format, so here we need some custom code to import it. This is available in the `lecroy.py` file.

A better way to store large amounts of data is in the commonly used [HDF5 binary data format](https://omni.wikiwand.com/en/articles/Hierarchical_Data_Format). In Python, this can be used via the [h5py](https://www.h5py.org) package.

We will also look at processing the data. For calculating spectra, filtering time series, etc., we will use tools from `scipy.signal` and `scipy.fft`. For fitting data to theoretical models, we will use tools from `scipy.optimize`.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import lecroy
import h5py
import os
import sys
from scipy.signal import periodogram, welch
from scipy.fft import rfft, irfft, rfftfreq
from scipy.optimize import curve_fit, least_squares

In [ ]:
jens_folder = 'test_data/jens'
jens_files = os.listdir(jens_folder)
xueshi_folder = 'test_data/xueshi'
xueshi_files = os.listdir(xueshi_folder)

In [ ]:
def lin2db(lin):
    return 10 * np.log10(lin)

def db2lin(db):
    return 10**(db/10)

In [ ]:
Vsq = lambda x, eta, frel: 1 - 4*eta*x / ((1+x)**2 + frel**2)
Vasq = lambda x, eta, frel: 1 + 4*eta*x / ((1-x)**2 + frel**2)
Vrot = lambda x, eta, frel, theta: np.cos(theta*np.pi/180)**2 * Vsq(x, eta, frel) + np.sin(theta*np.pi/180)**2 * Vasq(x, eta, frel)

def Vrotfluct(x, eta, frel, theta, sigma_x=0, sigma_p=None):
    if sigma_p is None:
        sigma_p = sigma_x
        
    Vx = .5 * Vsq(x, eta, frel) * (1 + np.cos(2 * theta * np.pi/180) * np.exp(-2 * (sigma_x * np.pi/180)**2))
    Vp = .5 * Vasq(x, eta, frel) * (1 - np.cos(2 * theta * np.pi/180) * np.exp(-2 * (sigma_p * np.pi/180)**2))

    return Vx + Vp

## Jens data

In [ ]:
jens_files

We select the files containing the spectrum:

In [ ]:
sq_spec_files = [os.path.join(jens_folder, f) for f in sorted(jens_files) if 'SPEC' in f and '_S' in f]
asq_spec_files = [os.path.join(jens_folder, f) for f in sorted(jens_files) if 'SPEC' in f and '_A' in f]
elec_spec_file = os.path.join(jens_folder, 'SPEC_elec.csv')
vac_spec_file = os.path.join(jens_folder, 'SPEC_vac.csv')
sq_spec_files, asq_spec_files

Read data from the files. The first 45 lines of each file containt metadata, so we first look at these, but then skip them when reading the actual data.

In [ ]:
with open(elec_spec_file) as f:
    meta = [f.readline().strip().split(',') for i in range(45)]
for text in meta:
    if len(text) > 1:
        print(f'{text[0]:<25} : {text[1]}')
    else:
        print(text)

In [ ]:
electronic = np.genfromtxt(elec_spec_file, delimiter=',', skip_header=45)
vacuum = np.genfromtxt(vac_spec_file, delimiter=',', skip_header=45)
squeezing = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in sq_spec_files])
antisqueezing = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in asq_spec_files])

In [ ]:
electronic.shape, vacuum.shape, squeezing.shape, antisqueezing.shape

In [ ]:
f = electronic[:,0] / 1e6  # to MHz
electronic = electronic[:,1]
vacuum = vacuum[:,1]
squeezing = squeezing[:,:,1]
antisqueezing = antisqueezing[:,:,1]

As we saw from the "Y Axis Units" in the meta data, the spectrum is already given on a logarithmic scale. We also see this when plotting:

In [ ]:
plt.plot(f, electronic, c='k')
plt.plot(f, vacuum, c='C2')
for sq in squeezing:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing:
    plt.plot(f, asq, c='C1')

We are interested in the squeezing and antisqueezing noise variance _relative_ to the vacuum noise, so we normalise to this.
Before doing this, we can subtract the electronic noise which is common for all traces. When performing arithmetics on the spectra, we must first convert from logarithmic to linear scale:

In [ ]:
vacuum_corr_lin = db2lin(vacuum) - db2lin(electronic)
squeezing_norm = lin2db((db2lin(squeezing) - db2lin(electronic)) / vacuum_corr_lin)
antisqueezing_norm = lin2db((db2lin(antisqueezing) - db2lin(electronic)) / vacuum_corr_lin)

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

Let's now try to fit the model outlined at the top of this notebook to the measured data. Here, I've used `least_squares` from `scipy.optimize`, but `curve_fit` is a more common choice. An improved curve fitting tool is in the [LMFIT](https://lmfit.github.io/lmfit-py/) package.

First I define a function that takes as arguments the fitting parameters in a list (here $x$, $\eta$ and $f_{HWHM}$), the independent variable (here $f$), and the y-variable data (here the spectrum), and outputs the residuals between data and model. Then I test the fitting using the most anti-squeezed spectrum, using initial guesses of the parameters.

In [ ]:
def res_asq(params, freq, spec):
    x, eta, bw = params
    model = lin2db(Vasq(x, eta, freq/bw))
    residuals = spec - model
    return residuals
    

In [ ]:
test_spec = antisqueezing_norm[2]
least_squares(res_asq, [.5,.8,40], args=[f, test_spec])

The error is due to the spectrum containing NaN values. These can be removed by masking them out, after which the fit works:

In [ ]:
np.sum(res_asq([.5,.8,40], f, test_spec)**2)

In [ ]:
mask1 = ~np.isnan(test_spec)
res = least_squares(res_asq, [.5,.8,40], args=[f[mask1], test_spec[mask1]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit = res.x
plt.plot(f, lin2db(Vasq(x_fit, eta_fit, f/bw_fit)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

The fit is still very bad, as can be seen visually from the poor fit between data and model (green) and from the very large cost value of the fitting result. The poor fit is due to the strong signals around 40 MHz and 100 MHz and the overtone of the 40 MHz signal at 80 MHz. There's also some nasty stuff at very low frequencies. I therefore create a new mask that removes these frequencies from the data that goes into the fit:

In [ ]:
mask2 = ((f>2) & (f<35)) | ((f>45) & (f<75)) | ((f>85) & (f<95))
mask = mask1 & mask2
len(np.nonzero(mask1 & mask2)[0])

In [ ]:
res = least_squares(res_asq, [.5,.8,40], args=[f[mask], test_spec[mask]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit = res.x
plt.plot(f, lin2db(Vasq(x_fit, eta_fit, f/bw_fit)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

This was a really nice fit! The fitted parameters are in the `x` variable of the fitting result: `[ 7.463e-01  8.170e-01  3.622e+01]`, that is, $x=0.746$, $\eta=0.817$, $f_{HWHM}=36.2\text{ MHz}$.

Could we perhaps make it even better by using the more complex model with phase and phase fluctuation parameters? When adding more parameters, the optimisation problem becomes more ill-defined and "loose", so I will add some bounds to the parameters to help it along.

In [ ]:
res = least_squares(res_asq2, [.5,.8,40, 90, 0], bounds=([0,.5,10,80,0], [.99,1,50,100,10]), args=[f[mask], test_spec[mask]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit, theta_fit, sigma_fit = res.x
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, theta_fit, sigma_fit)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

This doesn't work very well. The fit looks OK'ish, but two of the parameters are hitting their bounds which is a clear indication of a bad fit. Seems to be too many parameters.

Next, I try to fit both the antisqueezing and squeezing for a given power at the same time. Since they're recorded with the same power and same experimental setup, the parameters $x$, $\eta$ and $f_{HWHM}$ should be identical. I also assume $\sigma$ to be identical (not necessarily a good assumption). To fit both curves simultaneously, one can simply concatenate values for both spectra and for both models:

In [ ]:
asq_spec = antisqueezing_norm[2]
sq_spec = squeezing_norm[2]

def res_asq3(params, freq, sq_spec, asq_spec):
    x, eta, bw, theta_sq, theta_asq, sigma = params
    sq_model = lin2db(Vrotfluct(x, eta, freq/bw, theta_sq, sigma))
    asq_model = lin2db(Vrotfluct(x, eta, freq/bw, theta_asq, sigma))
    model = np.concatenate([sq_model, asq_model])
    spec = np.concatenate([sq_spec, asq_spec])
    residuals = spec - model
    return residuals

In [ ]:
res = least_squares(res_asq3, [.5,.8,40, 0, 90, 0], 
                    bounds=([0,.5,10,-10,80,0], [.99,1,50,10,100,10]), 
                    args=[f[mask], sq_spec[mask], sq_spec[mask]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit, theta_sq_fit, theta_asq_fit, sigma_fit = res.x
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, theta_sq_fit, sigma_fit)), 'C3')
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, theta_asq_fit, sigma_fit)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

That was a very poor fit! Let's try to remove the phase parameters and go back to basics:

In [ ]:
asq_spec = antisqueezing_norm[2]
sq_spec = squeezing_norm[2]

def res_asq3(params, freq, sq_spec, asq_spec):
    x, eta, bw = params
    sq_model = lin2db(Vrotfluct(x, eta, freq/bw, 0, 0))
    asq_model = lin2db(Vrotfluct(x, eta, freq/bw, 0, 0))
    model = np.concatenate([sq_model, asq_model])
    spec = np.concatenate([sq_spec, asq_spec])
    residuals = spec - model
    return residuals

In [ ]:
res = least_squares(res_asq3, [.5,.8,40], 
                    bounds=([0,.5,10], [.99,1,50]), 
                    args=[f[mask], sq_spec[mask], sq_spec[mask]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit = res.x
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, 0, 0)), 'C3')
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, 90, 0)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

The fit works much better now with fewer parameters. Still, for some reason, the dual fit doesn't fit well for the highest-power traces. I suspect there might be an issue with the data, or the model fails to take into account some property of the experiment. The fit for the lowest-power spectra also doesn't look amazing:

In [ ]:
asq_spec = antisqueezing_norm[0]
sq_spec = squeezing_norm[0]

def res_asq3(params, freq, sq_spec, asq_spec):
    x, eta, bw = params
    sq_model = lin2db(Vrotfluct(x, eta, freq/bw, 0, 0))
    asq_model = lin2db(Vrotfluct(x, eta, freq/bw, 0, 0))
    model = np.concatenate([sq_model, asq_model])
    spec = np.concatenate([sq_spec, asq_spec])
    residuals = spec - model
    return residuals

In [ ]:
res = least_squares(res_asq3, [.5,.8,40], 
                    bounds=([0,.5,10], [.99,1,50]), 
                    args=[f[mask], sq_spec[mask], sq_spec[mask]])
res

In [ ]:
for sq in squeezing_norm:
    plt.plot(f, sq, c='C0')
for asq in antisqueezing_norm:
    plt.plot(f, asq, c='C1')

x_fit, eta_fit, bw_fit = res.x
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, 0, 0)), 'C3')
plt.plot(f, lin2db(Vrotfluct(x_fit, eta_fit, f/bw_fit, 90, 0)), 'C2')

plt.ylim(-9,16)
plt.xlim(0, 120)
plt.grid()

However, another issue might be that measurements are a bit unreliable at higher frequencies where the detector efficiency is lowm (as seen in the first raw spectrum at the top). These have a relatively large weight in the fit and seem to skew the fit in a way that the more accurate low-frequency values don't fit well. 

To test this, below I made a "manual fitting machine" using widgets for the parameters. The default parameters inserted here work quite well for the lowest power spectra -- both squeezing and anti-squeezing -- at low frequencies, while especially the squeezing is somewhat off at higher frequencies. But this looks already much better. When increasing `x` it also somewhat matches the middle-power anti-squeezing and the highest-power squeezing - not what one would expect. It could be that the power labels in the file names are not correct, or (perhaps more likely) that the detector has some saturation at high noise powers.

This manual fitting shows that there is still some massaging to do to make the optimiser give a more realistic fit.

In [ ]:
from ipywidgets import interact

@interact(x=(0.,.999,.01), eta=(0.,1.,.01), bw=(10.,100.,1.), 
          theta_sq_offset=(-45.,45.,.1), theta_asq_offset=(-45.,45.,.1), sigma=(0.,10.,.01))
def manual_fit(x=.43, eta=.9, bw=33., theta_sq_offset=0., theta_asq_offset=0., sigma=0.):

    for sq in squeezing_norm:
        plt.plot(f, sq, c='C0')
    for asq in antisqueezing_norm:
        plt.plot(f, asq, c='C1')
    
    plt.plot(f, lin2db(Vrotfluct(x, eta, f/bw, 90+theta_asq_offset, sigma)), 'C2')
    plt.plot(f, lin2db(Vrotfluct(x, eta, f/bw, 0+theta_sq_offset, sigma)), 'C2')
    
    plt.ylim(-9,16)
    plt.xlim(0, 120)
    plt.grid()

### Save to HDF5

It is a very good idea to store data in the HDF5 format. It can contain multiple datasets, metadata, attributes on each dataset, and it is quite fast to look into even GB-large files using some of the available HDF5 viewers or e.g. the H5Web extension to VS Code. Here we format the metadata from one of the traces (using `dtype='S'` as the default `'U'` datatype for NumPy string arrays does not work) and place it in a single .hdf5 file together with all the spectra extracted from the CSV files.

In [ ]:
meta_elec = np.genfromtxt(elec_spec_file, delimiter=',', skip_header=2, max_rows=42, dtype='S')
meta_elec

In [ ]:
meta_elec = {a[0]:a[1] for a in meta_elec}
meta_elec

In [ ]:
with h5py.File('squeezing_data.hdf5', 'w') as file:
    dset_spec = file.create_group('spectrum')
    dset_freq = dset_spec.create_dataset('frequencies', data=f)
    dset_elec = dset_spec.create_dataset('electronic', data=electronic)
    dset_vac = dset_spec.create_dataset('vacuum', data=vacuum)
    dset_sqz = dset_spec.create_dataset('squeezing', data=squeezing)
    dset_asqz = dset_spec.create_dataset('antisqueezing', data=antisqueezing)
    for key, val in meta_elec.items():
        dset_spec.attrs[key] = val

The data can also be compressed:

In [ ]:
with h5py.File('squeezing_data_zipped.hdf5', 'w') as file:
    dset_spec = file.create_group('spectrum')
    dset_freq = dset_spec.create_dataset('frequencies', data=f, chunks=True, compression='gzip')
    dset_elec = dset_spec.create_dataset('electronic', data=electronic, chunks=True, compression='gzip')
    dset_vac = dset_spec.create_dataset('vacuum', data=vacuum, chunks=True, compression='gzip')
    dset_sqz = dset_spec.create_dataset('squeezing', data=squeezing, chunks=True, compression='gzip')
    dset_asqz = dset_spec.create_dataset('antisqueezing', data=antisqueezing, chunks=True, compression='gzip')
    for key, val in meta_elec.items():
        dset_spec.attrs[key] = val

Reading HDF5 files is also easy - just access it like a file system:

In [ ]:
file = h5py.File('squeezing_data_zipped.hdf5', 'r')
list(file.keys())

In [ ]:
list(file['spectrum'].keys())

In [ ]:
spec = file['spectrum']
spec['frequencies'][:]

In [ ]:
spec = file['spectrum']
plt.plot(spec['frequencies'][:], spec['electronic'][:])
file.close()

## Xueshi data

Below I give just the minimum necessary code to load Xueshi's data files. Then it's your turn to play around with it.

In [ ]:
datasets = ['electronic', 'shotnoise', 'squeezing', 'antisqueezing']
channels = ['C1', 'C2', 'C3', 'C4']

def get_file_list(dataset, channel):
    folder = os.path.join(xueshi_folder, dataset, channel)
    fns = sorted(os.listdir(folder))
    fns = [os.path.join(folder, fn) for fn in fns]
    return fns

files = {ds:
           {ch:get_file_list(ds, ch) for ch in channels}
           for ds in datasets}

In [ ]:
meta, times, data = lecroy.read(files['shotnoise']['C1'][0], scale=False)
dt = meta['horiz_interval']
fs = 1/dt
N = len(data[0])
t = np.linspace(0, fs*N, N, endpoint=False)
f = rfftfreq(N, dt)
# meta

In [ ]:
meta

In [ ]:
for ds in datasets:
    data = np.array([[
        lecroy.read(f, scale=False)[2] for f in files[ds][ch]]
                     for ch in channels])
    data = data.squeeze()
    globals()[ds] = data


In [ ]:
electronic.shape, shotnoise.shape, squeezing.shape, antisqueezing.shape

In [ ]:
plt.plot(t, antisqueezing[0,0])

## Exercises

Find the Jens data here: https://www.dropbox.com/scl/fo/3hct1gtuo9la8ysoyhuk2/AAuzFp9D7BVOpwDPJKEN_F0?rlkey=0dkbj53brwayjnj72jnzemau6&dl=0

And Xueshi data here: https://www.dropbox.com/scl/fo/as7ek7z1nezg2urofje4h/AD8NKSYoHrOBiPaVM9FnJMk?rlkey=4qmt8zoeqpv1h13feffsd9ssg&dl=0

1. In Jens' data, load the "5MHz..." files and repackage them into an HDF5 file together with metadata and the power values obtained from the file names.
2. Reproduce the figure `power_scaling.png`: Calculate the noise variance for each of the traces and do a fit to the model for OPO squeezing variance.
3. Load all of Xueshi's data using `lecroy.read` and repackage them into an HDF5 file.
4. On the raw time traces of squeezing and anti-squeezing, try to create and apply a filter that can remove the strong oscillation at around 28 MHz. Look into e.g. `firwin`, `iirwin`, `lfilter` or other methods of `scipy.signal`.
5. Calculate the averaged spectra of the data. You can consider each of the (electronic, shotnoise, squeezing, antisqueezing) to have been acquired under constant conditions, so it is okay to average all the traces for each channel. Look into `rfft` and `rfftfreq` of `scipy.fft` and/or `periodogram` and `welch` of `scipy.signal`. Look both at spectra for the individual channels and the sum of the four channels.
6. Try to fit the model of OPO squeezing/anti-squeezing to the obtained spectra and extra fitted parameters for $x$, $\eta$, $f_{HWHM}$ and possibly $\theta$ or $\sigma$.